# Data Selection
Ce notebook constitue la première étape du projet d'extraction et de structuration des données relatives au Bénin à partir de la base de données ***GDELT (Global Database of Events, Language, and Tone)***.

L’objectif principal de cette étape est de sélectionner un ou plusieurs datasets pertinents en lien avec le projet, en suivant les étapes suivantes :

-   Explorer la base de données

-   Identifier les datasets pertinents pour le sujet d’étude

-   Consulter les tables contenues dans ces datasets

-   Sélectionner les tables qui feront l’objet d’une extraction pour l’analyse future

Cette phase exploratoire est essentielle pour garantir que les données extraites seront à la fois cohérentes, pertinentes et exploitables en vue d'analyses ultérieures.

In [1]:
%pip install --upgrade google-cloud-bigquery pandas pyarrow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from functions.initialize_client import initialize_bigquery_client
from functions.setup_authentication import setup_authentication

# Définir le chemin vers le fichier de clé d'authentification BigQuery
bigquery_key_path = "/home/pionner02/Pionner02 UlChris-Project/pionner02-ulchris-457521-key.json"
auth_env = "GOOGLE_APPLICATION_CREDENTIALS"

# Définir la variable d'environnement pour authentifier l'accès à Google Cloud avec la clé
setup_authentication(bigquery_key_path, auth_env)

# Initialisation le client BigQuery en utilisant la clé d'authentification configurée précédemment
project_id = "pionner02-ulchris-457521"
client = initialize_bigquery_client(project_id)

2025-04-23 13:00:20,944 - INFO - Clé d'authentification chargée dans GOOGLE_APPLICATION_CREDENTIALS : /home/pionner02/Pionner02 UlChris-Project/pionner02-ulchris-457521-key.json
2025-04-23 13:00:20,944 - INFO - Aucun identifiant explicite, utilisation des Application Default Credentials
2025-04-23 13:00:20,984 - INFO - Client BigQuery initialisé pour le projet pionner02-ulchris-457521


## GDELT Data Preview

Cette section procède à une prévisualisation des données disponibles dans la base ***GDELT***. L’objectif est d’avoir une vision d’ensemble des datasets existants, ainsi que du type d’informations qu’ils contiennent.

In [4]:
# Projet distant contenant les datasets à explorer
target_project = "gdelt-bq"

# Récupération de la liste des datasets disponibles dans le projet "gdelt-bg"
datasets = list(client.list_datasets(project=target_project))

# Vérification si la liste des datasets est vide
if not datasets:
    print("Aucun dataset trouvé dans le projet gdelt-bq.")
else:
    # Affichage du nombre total de datasets trouvés
    print(f"Nombre total de datasets trouvés dans GDELT : {len(datasets)}")

    # Affichage des informations sur chaque dataset
    print(f"Nom du projet Google Cloud : {target_project}")
    print("\n=== Informations sur les bases de données GDELT ===")
            
    for dataset in datasets:
        try:
            # Tentative de récupération des métadonnées complètes du dataset
            full_dataset = client.get_dataset(dataset.reference)
            
            # Affichage des détails complets du dataset
            print(f"\nDataset : {full_dataset.dataset_id}")
            print(f"ID complet : {full_dataset.full_dataset_id}")
            print(f"Date de création : {full_dataset.created}")
            print(f"Dernière modification : {full_dataset.modified}")
            print(f"Description : {full_dataset.description or 'Aucune'}")
            print(f"Étiquettes : {full_dataset.labels or 'Aucune'}")
            
        except Exception as e:
            print(f"Erreur lors de la récupération des informations pour le dataset {dataset.dataset_id} : {e}")

Nombre total de datasets trouvés dans GDELT : 8
Nom du projet Google Cloud : gdelt-bq

=== Informations sur les bases de données GDELT ===

Dataset : covid19
ID complet : gdelt-bq:covid19
Date de création : 2020-03-30 00:36:39.040000+00:00
Dernière modification : 2024-11-19 22:08:34.013000+00:00
Description : Aucune
Étiquettes : {'dg_data_confidentiality': 'public', 'dg_data_source': 'public'}

Dataset : extra
ID complet : gdelt-bq:extra
Date de création : 2014-12-03 05:22:28.037000+00:00
Dernière modification : 2024-11-19 22:09:01.418000+00:00
Description : Aucune
Étiquettes : {'dg_data_source': 'public', 'dg_data_confidentiality': 'public'}

Dataset : full
ID complet : gdelt-bq:full
Date de création : 2014-04-23 23:04:48.708000+00:00
Dernière modification : 2024-11-19 22:05:19.124000+00:00
Description : Aucune
Étiquettes : {'dg_data_confidentiality': 'public', 'dg_data_source': 'public'}

Dataset : gdeltv2
ID complet : gdelt-bq:gdeltv2
Date de création : 2015-02-18 20:37:17.892000+00

In [5]:
print(f"GDELT regroupe {len(datasets)} datasets.")

# Liste des IDs des datasets cibles
target_datasets = [
                "extra",
                "full",
                "gdeltv2",
                "gdeltv2_ngrams",
                "sample_views"
                ]

print(f"Parmi lesquels, {len(target_datasets)} datasets paraissent être les plus pertinants pour ce projet :")

# Affichage des details des datasets dans target_datasets
for dataset in datasets:
    if dataset.dataset_id not in target_datasets:
        continue
    
    try:
        # Récupération des métadonnées complètes du dataset
        target_dataset = client.get_dataset(dataset.reference)
        
        # Affichage des détails complets du dataset
        print(f"\nDataset : {full_dataset.dataset_id}")  # ID du dataset (nom simple du dataset)
        print(f"ID complet : {full_dataset.full_dataset_id}")  # ID complet, qui inclut le projet et le dataset
        print(f"Date de création : {full_dataset.created}")  # Date de création du dataset
        print(f"Dernière modification : {full_dataset.modified}")  # Date de la dernière modification du dataset
        print(f"Description : {full_dataset.description or 'Aucune'}")  # Description du dataset, s'il y en a une, sinon 'Aucune'
        print(f"Étiquettes : {full_dataset.labels or 'Aucune'}")  # Liste des étiquettes associées au dataset, ou 'Aucune' si aucune étiquette n'est définie
        
    except Exception as e:
        print(f"Erreur lors de la récupération des informations pour le dataset {dataset.dataset_id} : {e}")        


GDELT regroupe 8 datasets.
Parmi lesquels, 5 datasets paraissent être les plus pertinants pour ce projet :



Dataset : sample_views
ID complet : gdelt-bq:sample_views
Date de création : 2014-04-24 04:50:03.517000+00:00
Dernière modification : 2024-11-19 22:05:56.624000+00:00
Description : Aucune
Étiquettes : {'dg_data_source': 'public', 'dg_data_confidentiality': 'public'}

Dataset : sample_views
ID complet : gdelt-bq:sample_views
Date de création : 2014-04-24 04:50:03.517000+00:00
Dernière modification : 2024-11-19 22:05:56.624000+00:00
Description : Aucune
Étiquettes : {'dg_data_source': 'public', 'dg_data_confidentiality': 'public'}

Dataset : sample_views
ID complet : gdelt-bq:sample_views
Date de création : 2014-04-24 04:50:03.517000+00:00
Dernière modification : 2024-11-19 22:05:56.624000+00:00
Description : Aucune
Étiquettes : {'dg_data_source': 'public', 'dg_data_confidentiality': 'public'}

Dataset : sample_views
ID complet : gdelt-bq:sample_views
Date de création : 2014-04-24 04:50:03.517000+00:00
Dernière modification : 2024-11-19 22:05:56.624000+00:00
Description : Aucune
Étiquet

## GDELT Dataset selection

Cette étape vient en amont de la prévisualisation des données ***GDELT***. Elle s'accorde à la sélection des datasets les plus pertinents pour répondre aux objectifs du projet.

In [6]:
# Affichage des tables par dataset préselectionné
print("\n=== Liste des tables par dataset préselectionné ===")

# Affichage des tables contenues dans target_datasets
for dataset in datasets:
    if dataset.dataset_id not in target_datasets:
        continue
    
    try:
        # Liste les tables disponibles dans ce dataset
        tables = list(client.list_tables(dataset.reference))
        # Affiche le nom du dataset et le nombre total de tables contenus dans target_datasets
        print(f"\nTables dans {dataset.dataset_id} ({len(tables)} tables) :")

        # Affiche les noms de toutes les tables
        if not tables:
            print("  Aucune table trouvée.")
        else:
            for table in tables:
                print(f"  - {table.table_id}")  # Nom de chaque table du dataset
                
    except Exception as e:
        print(f"Erreur lors de la récupération des tables pour le dataset {dataset.dataset_id} : {e}")


=== Liste des tables par dataset préselectionné ===

Tables dans extra (6 tables) :
  - countries_by_media_50pct
  - countrygeolookup
  - countryinfo
  - countryinfo2
  - sourcesbycountry
  - toytonelookup

Tables dans full (3 tables) :
  - crosswalk_geocountrycodetohuman
  - events
  - events_partitioned

Tables dans gdeltv2 (62 tables) :
  - academicliteraturegkg
  - cloudvision
  - cloudvision_partitioned
  - cloudvision_webentitygraph_20191020
  - domainsbycountry_alllangs_april2015
  - eventmentions
  - eventmentions_partitioned
  - events
  - events_partitioned
  - gal
  - gdg_partitioned
  - geg_g1
  - geg_gcnlapi
  - geg_gcnlapisent
  - geg_iatv
  - gegv2_iatv
  - gemg
  - gfg_linguistic1grams_20190831
  - gfg_linguistic2grams_20190831
  - gfg_linguisticinventory_20190901
  - gfg_outlet1grams_20190831
  - gfg_outlet2grams_20190831
  - gfg_partitioned
  - ggg
  - gkg
  - gkg_partitioned
  - gkg_socialoutlinks
  - gng
  - gqg
  - grg_dtn
  - grg_vcn
  - grg_vcn_nlpapi
  - gsg
  

Parmi les datasets proposés par ***GDELT***,  les datasets **full** et **gdeltv2**  ont été retenu sur la base de sa pertinence par rapport aux informations recherchées. En effet, elles proposent des tables dont les données répondent à la problématique de ce projet.

In [ ]:
# Liste des IDs des datasets cibles
target_datasets =   [
                    "full",
                    "gdeltv2"
                    ]

# Affichage de la structure des tables de chaque dataset
print(f"Schéma des tables pour le dataset {target_datasets}")

# Affichage des tables contenues dans target_datasets
for dataset in datasets:
    if dataset.dataset_id not in target_datasets:
        continue
    
    try:
        # Liste toutes les tables présentes dans le dataset actuel
        tables = list(client.list_tables(dataset.reference))

        # Affichage du nom du dataset analysé et du nombre de tables présentes
        print(f"\nTables dans le dataset '{dataset.dataset_id}' ({len(tables)} table(s)) :")

        # Affichage des informations de chaque table
        for table in tables:
            print(f"\n- Table : {table.table_id}")

            try:
                # Récupèration des métadonnées complètes de la table
                table_ref = client.get_table(table.reference)

                # Affichage de le structure de la table : noms des colonnes et types de données
                print("  Schéma de la table :")
                for field in table_ref.schema:
                    print(f"    - {field.name} ({field.field_type})")

            except Exception as e:
                print(f"    Erreur lors de la récupération du schéma de la table '{table.table_id}': {e}")
                continue

    except Exception as e:
        print(f"Erreur lors de la récupération des tables pour le dataset '{dataset.dataset_id}': {e}")

Schéma des tables pour une sélection de datasets

Tables dans le dataset 'gdeltv2' (62 table(s)) :

- Table : academicliteraturegkg
  Schéma de la table :
    - GKGRECORDID (STRING)
    - DATE (INTEGER)
    - SourceCollectionIdentifier (INTEGER)
    - SourceCommonName (STRING)
    - DocumentIdentifier (STRING)
    - Counts (STRING)
    - V2Counts (STRING)
    - Themes (STRING)
    - V2Themes (STRING)
    - Locations (STRING)
    - V2Locations (STRING)
    - Persons (STRING)
    - V2Persons (STRING)
    - Organizations (STRING)
    - V2Organizations (STRING)
    - V2Tone (STRING)
    - Dates (STRING)
    - GCAM (STRING)
    - SharingImage (STRING)
    - RelatedImages (STRING)
    - SocialImageEmbeds (STRING)
    - SocialVideoEmbeds (STRING)
    - Quotations (STRING)
    - AllNames (STRING)
    - Amounts (STRING)
    - TranslationInfo (STRING)
    - Extras (STRING)

- Table : cloudvision
  Schéma de la table :
    - DATE (INTEGER)
    - DocumentIdentifier (STRING)
    - ImageURL (STRING)

##  GDELT Tables Selection

Suite à la sélection des datasets **full** et **gdeltv2**, afin d’identifier les tables répondant le mieux aux besoins du projet, deux tables ont été retenues de l'exploration des tables menée. Le choix final du dataset approprié au projet se porte sur **gdeltv2** dont les tables `events_partitioned` et `eventmentions_partitioned`feront l'objet des analyses.

-   `events_partitioned` 

Elle est un repertoire détaillé des événements mondiaux. Elle fournit des informations clés telles que les dates, les types d’événements. C’est la table principale pour identifier et analyser les événements impliquant le Bénin.

-   `eventmentions_partitioned`

Cette table complète les données d’events_partitioned en listant les mentions médiatiques associées à chaque événement. Elle permet de mesurer l’ampleur médiatique d’un événement, le type de source, et d’autres métadonnées importantes.